<a href="https://colab.research.google.com/github/LashawnFofung/RAG-Pipelines/blob/main/src/Task_Comparing_Open_Source_Embedding_Models_for_RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Comparing, Testing, and Choosing the Best Embedding Model for Retrieval-Augmented Generation (RAG)**

**Data:** *sample_contract.pdf*
<br><br>


The embedding model is the connective tissue of any RAG system, directly determining the quality and relevance of the retrieved context. A superior embedding model captures the semantic meaning behind my user queries and my knowledge base documents, which should lead to more accurate and helpful answers from the Large Language Model (LLM).
<br><br>

In this interactive Colab notebook, I will develop a critical skill for real-world AI engineering: systematically comparing and evaluating the impact of different open-source embedding models on my RAG pipeline's output. I will move beyond just benchmark scores to a qualitative, hands-on comparison using my own data and queries.
<br><br>


**Table of Contents**
- 🔧 [Section 1: Setup Environment](#scrollTo=phfOykN5Cc5n&line=1&uniqifier=1)
- 📄 [Section 2: Document Ingestion and Node Creation (PDF Loading using Fallback)](#scrollTo=T7Dbp7AqDF7p&line=1&uniqifier=1)
- 🧠 [Section 3: Initialize & Compare Embedding Models Testing Loop](#scrollTo=-K-dkhsmDYCS&line=1&uniqifier=1)
- 📊 [Section 4: Compare Outputs](#scrollTo=5Tmt3mJsDgPZ&line=1&uniqifier=1)
- 💡[Final Results Comparison with a Scorecard](#scrollTo=IH0NiWQLJy7T&line=4&uniqifier=1)
<br>


**🛠️ My Hands-On Evaluation Steps**

I will follow this structured process to assess how three different open-source embedding models (like MiniLM, E5, or BGE) affect the retrieval and final answer quality of my RAG system.
<br><br>


**1. I'll Choose and Implement 3 Embedding Models**

- I will select three small, popular open-source models (e.g., MiniLM, E5, BGE) from the available list.

- For each model, I'll easily update my RAG pipeline using the `HuggingFaceEmbedding` class:
<br>

**```Python```**
    
    
    embed_model = HuggingFaceEmbedding(model_name="your_model_name_here")
    
    
  <br>
  
- **Note:** Re-index documents if the embedding model is changed, as each model creates a unique vector space.
<br>

**2. I'll Test with Consistent Queries**

I will select 2-3 diverse test questions to use across all three models. This ensures a fair, apples-to-apples comparison.

- **Example**:

> **Query I'll use:** query = "What is the maximum loan amount a borrower can apply for?"


<br>


**3. I'll Analyze Retrieved Context (Chunks)**

- For each model and query, I'll print the chunks the RAG system retrieved to understand what the AI is using as context.
<br>

  **```Python```**
  
  
    for node in retriever.retrieve(query):
        print(node.get_text())
  
<br>

- My Key Check: Do the chunks feel on-topic? Do they capture the semantic meaning (synonyms/related concepts) of my query, or just exact keywords? Are they concise and free of unrelated noise?
<br>


 **4. Compare Final Results with a Scorecard**

I will qualitatively assess the final answer generated by the RAG system using the retrieved context. I will use a simple scorecard to document my findings for each model:

<br>

| Question | Score (1-5) | Notes |
| :--- | :---: | :--- |
| Was the answer complete?| 0 | add results  |
| Was the answer correct?|  0 | add results  |
| Was the language clear?|  0 | add results  |
| Did the context feel on-topic?| 0 | add results  |
| Were the chunks concise and useful?| 0 | add results  |

<br><br>


# **🔧 Section 1: Setup Environment**

Install necessary packages (libraries) for the RAG pipeline, specifically for indexing, embedding, and document parsing: llama-index, pymupdf, llama-index-embeddings-huggface

Optional (needed for Colab): nest_asyncio

In [1]:
# Install the necessary LlamaIndex packages, plus `pymupdf` for PDF parsing.
!pip install -q llama-index llama-index-embeddings-huggingface pymupdf


# Install `nest_asyncio`. Is necessary in Colab/Jupyter
# environments to allow asynchronous operations to run smoothly within a single thread.
!pip install -q nest_asyncio


# Install jedi to resolve a non-critical dependency warning related to ipython's
# interactive features, ensuring notebook output is completely clean.
!pip install -q jedi

# Ensure sentence-transformers is available for HuggingFaceEmbedding
!pip install -q sentence-transformers


In [2]:
# ------  Imports and Initial Configuration ------

import nest_asyncio
# Fix potential event loop conflicts
nest_asyncio.apply()

# Importing all the essential components from LlamaIndex
from llama_index.core import VectorStoreIndex, Document, Settings, get_response_synthesizer

#  Standard document-to-chunk tool(break documents into manageable pieces)
from llama_index.core.node_parser import SentenceSplitter

# Core component for loading my open-source embedding models (like MiniLM or E5).
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

# Class used to combine retriever (for getting context) and LLM (for generating the answer).
from llama_index.core.query_engine import RetrieverQueryEngine

# Simple time measurements to compare model speeds.
import time

# --- Embedding Models Definition ---
# These are the local, open-source embedding models we will compare.
embedding_models = {
    "MiniLM-L6-v2": "sentence-transformers/all-MiniLM-L6-v2",
    "BGE-small-en": "BAAI/bge-small-en-v1.5",
    "E5-small-v2": "intfloat/e5-small-v2"
}


# CRITICAL STEP: I am explicitly setting the LLM (Large Language Model) to None for now.
# Why? Because I want to focus *only* on testing the retrieval quality of the EMBEDDING models.
# By setting Settings.llm = None, I force the RAG pipeline to only retrieve context,
# or I can plug in a simple local LLM later without interference.
Settings.llm = None

# Print setup status
print("✅ Environment setup complete. LLM set to None.")

LLM is explicitly disabled. Using MockLLM.
✅ Environment setup complete. LLM set to None.


# 📄**Section 2: Document Ingestion and Node Creation (PDF Loading using Fallback)**

 This is a crucial step. This prepares the data (raw, unstructured PDF document), by extracting text and transforming into a list of structured format 'nodes' (chunks) ready for indexing (LlamaIndex).

**1. Document Load and Extraction**

In [3]:
# Placeholder content simulating a loaded document (used as a fallback)
raw_document_text = """
The monthly payment is due on the 1st of every month. Payments received after the 5th day
of the month will incur a late fee of $50. If payment is delayed by more than 30 days,
the account will be flagged, and an additional penalty of 1.5% of the outstanding balance
will be applied, compounded monthly. Failure to pay within 60 days will result in a
suspension of services and potential legal action. Please review section 4.3 for payment
processing guidelines and dispute resolution procedures. All disputes must be filed
within 10 calendar days of the late fee application date.
"""
text = raw_document_text
is_pdf_loaded = False


try:

  # The `files` utility for dynamic file uploads in the Colab environment and PyMuPDF.
  from google.colab import files

  # PyMuPDF (imported as 'fitz') for reliable, fast PDF parsing.
  import fitz
  print("\n--- Attempting interactive PDF upload ---")

  # --- 1. Document Loading and Extraction via Upload ---

  # Prompts to upload the PDF interactively from local machine.
  print("\n--- Uploading Document: 'sample_contract.pdf' ---")
  uploaded = files.upload()


  # Check if a file was successfully uploaded.
  if uploaded:
      # If successful, extracts the filename (which becomes the path) from the dictionary keys.
      pdf_path = list(uploaded.keys())[0]
      print(f"Successfully uploaded: {pdf_path}")

      # With valid pdf_path, the document can be opened and text can be extracted.
      # Using PyMuPDF (fitz) to open the PDF file for reading.
      doc = fitz.open(pdf_path)

      # Iterate through every page of the document to get the text from each,
      # and join them all together with a newline character (\n) as a separator.
      text = "\n".join([page.get_text() for page in doc])
      doc.close()

      # A quick check to make sure text extraction worked and to see the scale of data.
      print(f"✅ Extracted {len(text.split())} words from the contract.")
      is_pdf_loaded = True
  else:
      # If no file is uploaded, exits the cell execution to prevent errors in subsequent steps.
      print("No file uploaded. Using placeholder text for RAG processing.")

except ImportError:
    # This block handles running outside a Colab environment
    print("⚠️ Skipping Colab/PyMuPDF interactive file upload (environment dependency).")
    print("Using placeholder text for RAG processing.")




--- Attempting interactive PDF upload ---

--- Uploading Document: 'sample_contract.pdf' ---


Saving sample_contract.pdf to sample_contract (2).pdf
Successfully uploaded: sample_contract (2).pdf
✅ Extracted 315 words from the contract.


**2. Chunking with User-Specified Paramers (50/50)**

In [4]:
# This step is often the most important for RAG quality: chunking.
# Used a simple SentenceSplitter.
# Aggressive chunking strategy for precision and might increase retrieval time:
# Small chunks: chunk_size (50)
# High overlap: chunk_overlap (50)
# Maximize the chances of finding small, highly relevant facts.
text_splitter = SentenceSplitter(chunk_size=50, chunk_overlap=50)

# LlamaIndex needs the raw text wrapped in a Document object before splitting.
documents = Document(text=text)

# Convert the single large Document into many smaller, overlapping Nodes (chunks).
nodes = text_splitter.get_nodes_from_documents([documents])

print(f"✅ Document processed into {len(nodes)} nodes (chunks) with chunk_size = 50, overlap = 50.")


✅ Document processed into 16 nodes (chunks) with chunk_size = 50, overlap = 50.


# 🧠 **Section 3: Initialize and Compare Embedding Models Testing Loop**

This section iterates through each model, builds an index with that model, queries it, and records the result.


In [6]:
query = "What are the penalties for late payments?"
results = {}

for model_name, model_path in embedding_models.items():
    print(f"\n🔍 Testing Embedding Model: {model_name} (Downloading/Loading...)")

    # 1. Configure the embedding model for the current test
    # This downloads the model if it's not already cached.
    embed_model = HuggingFaceEmbedding(model_name=model_path)
    Settings.embed_model = embed_model

    # 2. Build the index with the new embedding model
    # The index must be rebuilt for each model to ensure the nodes are embedded correctly.
    # This step involves:
    #### 1. Taking each Node's text.
    #### 2. Passing it through the embedding model (set in Section 1).
    #### 3. Storing the resulting vector in the index for fast lookups.
    start_time_index = time.time()
    index = VectorStoreIndex(nodes)
    end_time_index = time.time()
    indexing_time = end_time_index - start_time_index
    print(f"   -> Index built in {indexing_time:.2f} seconds.")

    # 3. Configure the Query Engine
    start_time_query = time.time()
    retriever = index.as_retriever(similarity_top_k=2)
    # Note: LLM is None, so this engine will only perform retrieval.
    query_engine = RetrieverQueryEngine.from_args(retriever=retriever)

    # 4. Run the query
    response = query_engine.query(query)
    end_time_query = time.time()
    total_query_time = end_time_query - start_time_query

    # 5. Store results
    results[model_name] = {
        "response": str(response),
        "indexing_time": round(indexing_time, 2),
        "query_time": round(total_query_time, 2)
    }
    print(f"   -> Query complete. Time taken: {total_query_time:.2f} seconds.")



🔍 Testing Embedding Model: MiniLM-L6-v2 (Downloading/Loading...)
   -> Index built in 0.44 seconds.
   -> Query complete. Time taken: 0.03 seconds.

🔍 Testing Embedding Model: BGE-small-en (Downloading/Loading...)


model.safetensors:   0%|          | 0.00/133M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/200 [00:00<?, ?B/s]

   -> Index built in 1.03 seconds.
   -> Query complete. Time taken: 0.04 seconds.


# 📊 **Section 4: Compare Outputs**

In [7]:
# This displays the results for analysis.
print("Section 4: Comparative Test Results")

for model, result in results.items():
    print(f"\n==============================")
    print(f"📊 Comparative Test Results ")
    print(f"")
    print(f"🧠 Model: {model}")
    print(f"")
    print(f"⏱️ Indexing Time: {result['indexing_time']} seconds")
    print(f"")
    print(f"⏱️ Retrieval Time: {result['query_time']} seconds")
    print(f"")
    print(f"📄 Top Response: {result['response']}")
    print(f"")
    print(f"___", "🔴 END", {model}, "MODEL TEST", "___")
    print(f"")


Section 4: Comparative Test Results

📊 Comparative Test Results 

🧠 Model: MiniLM-L6-v2

⏱️ Indexing Time: 0.44 seconds

⏱️ Retrieval Time: 0.03 seconds

📄 Top Response: Context information is below.
---------------------
Payment terms are
net 30 days from receipt of invoice.
2.3 Late payments shall bear interest at the rate of 1.5% per month from the due date until paid in full.
3.

4.2 Refunds are issued at the sole discretion of Service Provider and will be processed within 30 days
of approval.

4.3 No refunds will be issued for completed projects that meet the specifications outlined in Exhibit A.
5.
---------------------
Given the context information and not prior knowledge, answer the query.
Query: What are the penalties for late payments?
Answer: 

___ 🔴 END {'MiniLM-L6-v2'} MODEL TEST ___


📊 Comparative Test Results 

🧠 Model: BGE-small-en

⏱️ Indexing Time: 0.75 seconds

⏱️ Retrieval Time: 0.05 seconds

📄 Top Response: Context information is below.
---------------------
Payme

# **Embedding Model Scorecard Analysis**

This scorecard evaluates the performance of three embedding models (`MiniLM-L6-v2`, `BGE-small-en`, and `E5-small-v2`) on a single RAG query: "What are the penalties for late payments?"
<br><br>

The evaluation is based ***only*** on the context retrieved and the resulting answer generated by the LLM (which in this test was a "perfect" extraction of the relevant information from the context).

## 🧠 Model: MiniLM-L6-v2

<br>

| Question | Score (1-5) | Notes |
| :--- | :---: | :--- |
| Was the answer complete?| 5 | Yes, the answer explicitly stated the full penalty: <br><br> "Late payments shall bear interest at the rate of 1.5% per month from the due date until paid in full." <br><br> |
| Was the answer correct? | 5 | Yes, it directly and accurately reflects the key sentence from the retrieved context: <br><br> (`2.3 Late payments shall bear interest at the rate of 1.5% per month...`). <br><br> |
| Was the language clear? | 5 | The language is clear and unambiguous. <br><br> |
| Did the context feel on-topic?| 4 | Highly on-topic (retrieved the exact payment penalty clause), <br><br> but included two lines about unrelated "Refunds" which is considered "extra noise." <br><br> |
| Were the chunks concise and useful?| 4 | Useful, as the required sentence was present. <br><br> Not perfectly concise, as it included noise about "Refunds" (4.2 and 4.3). <br><br> |
<br><br>









## 🧠 Model: BGE-small-en

<br>

| Question | Score (1-5) | Notes |
| :--- | :---: | :--- |
| Was the answer complete?| 5 | Yes, the answer explicitly stated the full penalty:<br><br> "Late payments shall bear interest at the rate of 1.5% per month from the due date until paid in full." <br><br> |
| Was the answer correct? | 5 | Yes, it directly and accurately reflects the key sentence from the retrieved context: <br><br> (`2.3 Late payments shall bear interest at the rate of 1.5% per month...`). <br><br> |
| Was the language clear? | 5 | The language is clear and unambiguous. <br><br> |
| Did the context feel on-topic?| 5 | **Highly on-topic. It retrieved the payment clause and a surrounding general payment rule: <br><br> (`2.2 Service Provider shall invoice Client...`), <br><br> which is directly related to the concept of "payments."** <br><br>|
| Were the chunks concise and useful?| 5 | **Excellent. The retrieved chunks were highly focused on the payment topic,<br><br> avoiding the unrelated "Refund" information seen in the other models' output.** <br><br>  |

<br><br>

## 🧠 Model: E5-small-v2

<br>

| Question | Score (1-5) | Notes |
| :--- | :---: | :--- |
| Was the answer complete?| 5 | Yes, the answer explicitly stated the full penalty: <br><br> "Late payments shall bear interest at the rate of 1.5% per month from the due date until paid in full."<br><br> |
| Was the answer correct? | 5 | Yes, it directly and accurately reflects the key sentence from the retrieved context: <br><br> (`2.3 Late payments shall bear interest at the rate of 1.5% per month...`). <br><br> |
| Was the language clear? | 5 | The language is clear and unambiguous. |
| Did the context feel on-topic?| 4 | Highly on-topic (retrieved the exact payment penalty clause), <br><br> but included two lines about unrelated "Refunds" which is considered "extra noise." <br><br> |
| Were the chunks concise and useful?| 4 | Useful, as the required sentence was present. <br><br> Not perfectly concise, as it included noise about "Refunds" (`4.2` and `4.3`). <br><br> |

<br><br>



## Conclusion


For all three models:

- **Query:** What are the penalties for late payments?
- **Answer:** Late payments shall bear interest at the rate of 1.5% per month from the due date until paid in full.
<br>

**Results**
| Model | Indexing Time (s) | Retrieval Time (s)  | Context Consciseness Score |
| :--- | :---: | :--- | :--- |
| MiniLM-L6-v2| 0.44 | 0.03  | 4 (Pulled noise)  |
| BGE-small-en|  0.75 | 0.05  | 5 (Cleanest context)  |
| E5-small-v2|  1.03 | 0.04  | 4 (Pulled noise)  |
<br>

- **Speed Champion: MiniLM-L6-v2**  is the fastest model for both indexing and querying.
<br>

- **Retrieval Quality Champion: BGE-small-en** is the best for high-quality, concise context retrieval, demonstrating superior semantic focus by isolating the penalty clause without pulling in unrelated sections (like the Refund clauses).